# 🔥 KURE 기반 한국형 번아웃 감정 분류 모델
## AI Hub 데이터셋 활용 버전

**데이터**: AI Hub 감성대화 말뭉치 + 한국어 감정 정보 대화 데이터셋

**번아웃 카테고리 (박수정 외, 2018)**:
- 0: 정서적_고갈
- 1: 좌절_압박
- 2: 부정적_대인관계
- 3: 자기비하

## 1. 환경 설정

In [ ]:
!pip install -q sentence-transformers transformers torch pandas scikit-learn tqdm openpyxl

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
from typing import List, Dict, Optional, Tuple
from tqdm.auto import tqdm
import json
import os
import re
from collections import Counter

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🖥️ Device: {device}")
if device == "cuda":
    print(f"   GPU: {torch.cuda.get_device_name(0)}")

## 2. Google Drive 마운트 & 데이터 업로드

**방법 1**: Google Drive에 데이터 업로드 (권장)
- `dataset` 폴더를 Drive에 업로드

**방법 2**: 직접 업로드
- JSON 파일 직접 업로드

In [ ]:
# 방법 1: Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 데이터 경로 설정 (본인 경로에 맞게 수정)
# Drive에 dataset 폴더를 업로드했다면:
DATA_PATH = "/content/drive/MyDrive/Burnout/dataset"  # 본인 경로로 수정!

# 경로 확인
import os
if os.path.exists(DATA_PATH):
    print(f"✓ 데이터 경로 확인됨: {DATA_PATH}")
    print(f"  폴더 내용: {os.listdir(DATA_PATH)}")
else:
    print(f"❌ 경로를 찾을 수 없음: {DATA_PATH}")
    print("   Drive에 데이터를 업로드하거나 경로를 수정해주세요.")

In [ ]:
# 방법 2: 직접 업로드 (Drive 없이 사용 시)
# from google.colab import files
# uploaded = files.upload()  # JSON 파일 선택
# DATA_PATH = "/content"

## 3. 번아웃 카테고리 & 감정 매핑 정의

In [ ]:
# 번아웃 카테고리
BURNOUT_CATEGORIES = {
    0: "정서적_고갈",
    1: "좌절_압박",
    2: "부정적_대인관계",
    3: "자기비하"
}

# AI Hub 감정 타입(E코드) → 번아웃 카테고리 매핑
# 감성대화 말뭉치의 emotion type 코드 기반
EMOTION_TYPE_TO_BURNOUT = {
    # 정서적 고갈 (0) - 피로, 무기력, 우울 계열
    "E20": 0,  # 슬픔
    "E22": 0,  # 비통
    "E24": 0,  # 우울
    "E26": 0,  # 무기력
    "E40": 0,  # 우울/슬픔
    "E53": 0,  # 외로움
    "E32": 0,  # 스트레스
    
    # 좌절/압박 (1) - 분노, 짜증, 억울 계열
    "E10": 1,  # 분노/화남
    "E15": 1,  # 악의/증오
    "E18": 1,  # 화남/짜증
    "E19": 1,  # 성가심/귀찮음
    "E21": 1,  # 실망
    "E47": 1,  # 억울
    
    # 부정적 대인관계 (2) - 배신, 소외, 갈등 계열
    "E42": 2,  # 배신
    "E49": 2,  # 버림받음
    "E51": 2,  # 어색함/관계갈등
    
    # 자기비하 (3) - 불안, 걱정, 수치심 계열
    "E16": 3,  # 초조/조급
    "E25": 3,  # 마비/얼어붙음
    "E30": 3,  # 불안
    "E35": 3,  # 당혹
    "E37": 3,  # 걱정
    "E50": 3,  # 당황
    "E54": 3,  # 열등감
    "E56": 3,  # 부끄러움
    "E57": 3,  # 자기혐오
    "E58": 3,  # 자기비난
    
    # 긍정 감정 (제외: -1)
    "E62": -1,  # 신뢰
    "E64": -1,  # 만족
    "E66": -1,  # 편안
    "E67": -1,  # 안도
    "E68": -1,  # 기쁨
    "E69": -1,  # 자신감
}

# 키워드 기반 매핑 (백업)
KEYWORD_TO_BURNOUT = {
    # 정서적 고갈
    "지침": 0, "피곤": 0, "무기력": 0, "우울": 0, "슬픔": 0,
    "공허": 0, "외로": 0, "힘들": 0, "지쳐": 0, "스트레스": 0,
    
    # 좌절/압박
    "분노": 1, "화남": 1, "화가": 1, "짜증": 1, "억울": 1,
    "불만": 1, "실망": 1, "성가": 1, "귀찮": 1,
    
    # 부정적 대인관계
    "배신": 2, "갈등": 2, "무시": 2, "소외": 2, "서운": 2,
    "따돌": 2, "어색": 2,
    
    # 자기비하
    "불안": 3, "걱정": 3, "초조": 3, "두려": 3, "부끄러": 3,
    "창피": 3, "자책": 3, "당황": 3, "열등": 3,
}

print("✓ 매핑 테이블 정의 완료")

## 4. AI Hub 데이터 로더

In [ ]:
def clean_text(text: str) -> str:
    """텍스트 정제"""
    if not text:
        return ""
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)  # 반복 문자 정규화
    return text


def get_burnout_label(emotion_type: str, text: str = "") -> int:
    """
    감정 타입 코드와 텍스트를 기반으로 번아웃 레이블 반환
    Returns: 0-3 (번아웃 카테고리) 또는 -1 (제외)
    """
    # 1. emotion type 코드로 매핑 시도
    if emotion_type in EMOTION_TYPE_TO_BURNOUT:
        return EMOTION_TYPE_TO_BURNOUT[emotion_type]
    
    # 2. 키워드 기반 매핑
    text_lower = text.lower()
    for keyword, label in KEYWORD_TO_BURNOUT.items():
        if keyword in text_lower:
            return label
    
    return -1  # 매핑 실패


def load_emotional_dialogue_json(json_path: str) -> pd.DataFrame:
    """
    감성대화 말뭉치 JSON 로드
    AI Hub 형식: [{"profile": {..., "emotion": {"type": "E18", ...}}, "talk": {"content": {...}}}]
    """
    print(f"📂 Loading: {json_path}")
    
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    records = []
    
    for item in tqdm(data, desc="Parsing"):
        try:
            # emotion type 추출
            emotion_info = item.get('profile', {}).get('emotion', {})
            emotion_type = emotion_info.get('type', '')
            
            # 대화 내용 추출
            content = item.get('talk', {}).get('content', {})
            
            # 사용자 발화(HS01, HS02, HS03)만 추출
            for key in ['HS01', 'HS02', 'HS03']:
                text = content.get(key, '')
                if text and len(text) >= 10:  # 최소 길이 필터
                    text = clean_text(text)
                    burnout_label = get_burnout_label(emotion_type, text)
                    
                    records.append({
                        'text': text,
                        'emotion_type': emotion_type,
                        'burnout_label': burnout_label,
                        'source': '감성대화'
                    })
        except Exception as e:
            continue
    
    df = pd.DataFrame(records)
    print(f"✓ Loaded {len(df)} samples")
    return df


def load_continuous_dialogue_xlsx(xlsx_path: str) -> pd.DataFrame:
    """
    한국어 감정 대화 데이터셋 (xlsx) 로드
    """
    print(f"📂 Loading: {xlsx_path}")
    
    df = pd.read_excel(xlsx_path)
    print(f"  Columns: {df.columns.tolist()}")
    
    # 컬럼명에 따라 조정 필요
    # 일반적인 구조: ['발화', '감정', ...] 또는 ['text', 'emotion', ...]
    records = []
    
    # 실제 데이터 구조에 맞게 수정 필요
    text_col = None
    emotion_col = None
    
    for col in df.columns:
        if '발화' in col or 'text' in col.lower() or 'sentence' in col.lower():
            text_col = col
        if '감정' in col or 'emotion' in col.lower() or 'label' in col.lower():
            emotion_col = col
    
    if text_col and emotion_col:
        for _, row in df.iterrows():
            text = str(row[text_col])
            emotion = str(row[emotion_col])
            
            if text and len(text) >= 10:
                text = clean_text(text)
                burnout_label = get_burnout_label('', text)  # 키워드 기반
                
                records.append({
                    'text': text,
                    'emotion_type': emotion,
                    'burnout_label': burnout_label,
                    'source': '연속대화'
                })
    
    result_df = pd.DataFrame(records)
    print(f"✓ Loaded {len(result_df)} samples")
    return result_df


print("✓ 데이터 로더 함수 정의 완료")

## 5. 데이터 로드 & 전처리

In [ ]:
# 감성대화 말뭉치 로드
emotional_json_path = os.path.join(
    DATA_PATH,
    "018.감성대화/Training_221115_add/라벨링데이터/감성대화말뭉치(최종데이터)_Training/감성대화말뭉치(최종데이터)_Training.json"
)

if os.path.exists(emotional_json_path):
    df_emotional = load_emotional_dialogue_json(emotional_json_path)
else:
    print(f"❌ 파일을 찾을 수 없음: {emotional_json_path}")
    print("   경로를 확인해주세요.")
    df_emotional = pd.DataFrame()

In [ ]:
# (선택) 연속 대화 데이터셋 로드
continuous_xlsx_path = os.path.join(
    DATA_PATH,
    "한국어 감정 정보가 포함된 연속적 대화 데이터셋/한국어_연속적_대화_데이터셋.xlsx"
)

if os.path.exists(continuous_xlsx_path):
    df_continuous = load_continuous_dialogue_xlsx(continuous_xlsx_path)
else:
    print(f"파일 없음 (선택사항): {continuous_xlsx_path}")
    df_continuous = pd.DataFrame()

In [ ]:
# 데이터 합치기 & 필터링
dfs = [df for df in [df_emotional, df_continuous] if len(df) > 0]

if dfs:
    df_combined = pd.concat(dfs, ignore_index=True)
    print(f"\n📊 전체 데이터: {len(df_combined)}개")
    
    # 긍정 감정(-1) 제외
    df_filtered = df_combined[df_combined['burnout_label'] != -1].copy()
    print(f"   번아웃 관련 데이터: {len(df_filtered)}개")
    
    # 중복 제거
    df_filtered = df_filtered.drop_duplicates(subset=['text'])
    print(f"   중복 제거 후: {len(df_filtered)}개")
    
    # 분포 확인
    print("\n📈 번아웃 카테고리 분포:")
    for label, name in BURNOUT_CATEGORIES.items():
        count = len(df_filtered[df_filtered['burnout_label'] == label])
        pct = 100 * count / len(df_filtered) if len(df_filtered) > 0 else 0
        print(f"   {label}. {name}: {count}개 ({pct:.1f}%)")
else:
    print("❌ 로드된 데이터가 없습니다.")
    df_filtered = pd.DataFrame()

In [ ]:
# 샘플 데이터 확인
if len(df_filtered) > 0:
    print("\n📝 샘플 데이터:")
    for label in range(4):
        sample = df_filtered[df_filtered['burnout_label'] == label].head(2)
        print(f"\n[{BURNOUT_CATEGORIES[label]}]")
        for _, row in sample.iterrows():
            print(f"  - {row['text'][:50]}...")

## 6. Train/Val 분할

In [ ]:
from sklearn.model_selection import train_test_split

if len(df_filtered) > 100:
    train_df, val_df = train_test_split(
        df_filtered,
        test_size=0.1,
        stratify=df_filtered['burnout_label'],
        random_state=42
    )
    
    print(f"\n📊 데이터셋 분할:")
    print(f"   Train: {len(train_df)}개")
    print(f"   Val: {len(val_df)}개")
else:
    print("데이터가 부족합니다. 샘플 데이터로 진행합니다.")
    train_df = df_filtered
    val_df = None

## 7. 모델 정의

In [ ]:
class KUREEmbeddingLayer(nn.Module):
    def __init__(self, model_name="nlpai-lab/KURE-v1", freeze_encoder=True, device=None):
        super().__init__()
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        print(f"📥 Loading KURE model...")
        self.encoder = SentenceTransformer(model_name, device=self.device)
        self.embedding_dim = self.encoder.get_sentence_embedding_dimension()
        print(f"✓ Embedding dim: {self.embedding_dim}")
        if freeze_encoder:
            for param in self.encoder.parameters():
                param.requires_grad = False

    def forward(self, texts):
        return self.encoder.encode(texts, convert_to_tensor=True, show_progress_bar=False)


class BurnoutClassificationHead(nn.Module):
    def __init__(self, input_dim=1024, hidden_dim=256, num_classes=4, dropout=0.3):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, num_classes)
        )

    def forward(self, x):
        return self.classifier(x)


class KUREBurnoutClassifier(nn.Module):
    def __init__(self, model_name="nlpai-lab/KURE-v1", hidden_dim=256,
                 num_classes=4, dropout=0.3, freeze_encoder=True, device=None):
        super().__init__()
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.embedding_layer = KUREEmbeddingLayer(model_name, freeze_encoder, self.device)
        self.classification_head = BurnoutClassificationHead(
            self.embedding_layer.embedding_dim, hidden_dim, num_classes, dropout
        ).to(self.device)
        self.num_classes = num_classes

    def forward(self, texts):
        embeddings = self.embedding_layer(texts)
        return self.classification_head(embeddings)

    def predict(self, texts):
        self.eval()
        with torch.no_grad():
            logits = self.forward(texts)
            probs = F.softmax(logits, dim=-1)
            labels = torch.argmax(logits, dim=-1).cpu().tolist()
            categories = [BURNOUT_CATEGORIES[l] for l in labels]
        return labels, categories, probs

    def save_model(self, path):
        torch.save({'classification_head': self.classification_head.state_dict()}, path)
        print(f"✓ Saved: {path}")

    def load_model(self, path):
        ckpt = torch.load(path, map_location=self.device)
        self.classification_head.load_state_dict(ckpt['classification_head'])
        print(f"✓ Loaded: {path}")


print("✓ 모델 클래스 정의 완료")

## 8. Dataset & Training

In [ ]:
class BurnoutDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return {'text': self.texts[idx], 'label': self.labels[idx]}


def collate_fn(batch):
    return {
        'texts': [item['text'] for item in batch],
        'labels': torch.tensor([item['label'] for item in batch], dtype=torch.long)
    }


def train_model(model, train_dataset, val_dataset=None, epochs=10, batch_size=16, lr=2e-4):
    device = model.device
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn) if val_dataset else None

    optimizer = torch.optim.AdamW(model.classification_head.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    history = {'train_loss': [], 'train_acc': [], 'val_acc': []}

    for epoch in range(epochs):
        model.train()
        total_loss, correct, total = 0, 0, 0

        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
        for batch in pbar:
            texts, labels = batch['texts'], batch['labels'].to(device)
            optimizer.zero_grad()
            logits = model(texts)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, pred = torch.max(logits, 1)
            total += labels.size(0)
            correct += (pred == labels).sum().item()
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})

        train_acc = 100 * correct / total
        history['train_loss'].append(total_loss / len(train_loader))
        history['train_acc'].append(train_acc)

        # Validation
        val_acc = 0
        if val_loader:
            model.eval()
            val_correct, val_total = 0, 0
            with torch.no_grad():
                for batch in val_loader:
                    texts, labels = batch['texts'], batch['labels'].to(device)
                    logits = model(texts)
                    _, pred = torch.max(logits, 1)
                    val_total += labels.size(0)
                    val_correct += (pred == labels).sum().item()
            val_acc = 100 * val_correct / val_total
            history['val_acc'].append(val_acc)

        print(f"  Loss: {history['train_loss'][-1]:.4f} | Train: {train_acc:.1f}% | Val: {val_acc:.1f}%")

    return model, history


print("✓ Training 함수 정의 완료")

## 9. 모델 학습 🚀

In [ ]:
# 모델 초기화
model = KUREBurnoutClassifier(
    model_name="nlpai-lab/KURE-v1",
    hidden_dim=256,
    num_classes=4,
    dropout=0.3,
    freeze_encoder=True
)

In [ ]:
# Dataset 생성
if len(train_df) > 0:
    train_dataset = BurnoutDataset(
        train_df['text'].tolist(),
        train_df['burnout_label'].tolist()
    )
    
    val_dataset = None
    if val_df is not None and len(val_df) > 0:
        val_dataset = BurnoutDataset(
            val_df['text'].tolist(),
            val_df['burnout_label'].tolist()
        )
    
    print(f"✓ Train: {len(train_dataset)}개")
    print(f"✓ Val: {len(val_dataset) if val_dataset else 0}개")
else:
    print("❌ 데이터가 없습니다.")

In [ ]:
# 학습 실행
if len(train_df) > 0:
    model, history = train_model(
        model=model,
        train_dataset=train_dataset,
        val_dataset=val_dataset,
        epochs=10,
        batch_size=32,
        lr=2e-4
    )

## 10. 모델 저장

In [ ]:
# Google Drive에 저장
save_path = "/content/drive/MyDrive/Burnout/burnout_model.pt"
model.save_model(save_path)

## 11. 테스트

In [ ]:
test_texts = [
    "오늘도 야근이다. 너무 지치고 힘들다.",
    "상사가 또 화를 냈다. 억울하고 분하다.",
    "팀원들이 나를 무시하는 것 같다.",
    "나는 왜 이것밖에 못하는 걸까. 불안하다.",
    "일은 왜 해도 해도 끝이 없을까? 화가 난다.",
]

labels, categories, probs = model.predict(test_texts)

print("\n" + "="*60)
print("🔮 예측 결과")
print("="*60)
for text, cat, prob in zip(test_texts, categories, probs):
    print(f"\n입력: {text}")
    print(f"예측: {cat}")
    prob_str = {k: f"{v:.1%}" for k, v in zip(BURNOUT_CATEGORIES.values(), prob.cpu().numpy())}
    print(f"확률: {prob_str}")

## 12. 커스텀 테스트

In [ ]:
my_text = "월요일 아침부터 야근 예고를 받았다. 벌써 지친다."  # @param {type:"string"}

labels, categories, probs = model.predict([my_text])

print(f"\n입력: {my_text}")
print(f"\n🏷️ 예측: {categories[0]}")
print(f"\n📊 확률:")
for name, p in zip(BURNOUT_CATEGORIES.values(), probs[0].cpu().numpy()):
    bar = "█" * int(p * 20)
    print(f"  {name}: {bar} {p:.1%}")